# Rede Neural: Classificação Versão 3

Esta versão serve para testar um dataset maior na aplicação da técnica de redes neurais.
O dataset escolhido tem 63KB


*   Conjunto de treinamento: 80%
*   Conjunto de teste: 20%

Sobre o dataset:
Simula o registro de alta energia de partículas gama em um telescópio gama atmosférico Cherenkov usando técnica de imagem.

Informações sobre os atributos:
 1.  fLength:  continuous  # major axis of ellipse [mm]
    2.  fWidth:   continuous  # minor axis of ellipse [mm] 
    3.  fSize:    continuous  # 10-log of sum of content of all pixels [in #phot]
    4.  fConc:    continuous  # ratio of sum of two highest pixels over fSize  [ratio]
    5.  fConc1:   continuous  # ratio of highest pixel over fSize  [ratio]
    6.  fAsym:    continuous  # distance from highest pixel to center, projected onto major axis [mm]
    7.  fM3Long:  continuous  # 3rd root of third moment along major axis  [mm] 
    8.  fM3Trans: continuous  # 3rd root of third moment along minor axis  [mm]
    9.  fAlpha:   continuous  # angle of major axis with vector to origin [deg]
   10.  fDist:    continuous  # distance from origin to center of ellipse [mm]
   11.  class:    g,h         # gamma (signal), hadron (background)



In [0]:
import numpy as np
import pandas as pd
from sklearn import datasets

In [15]:
from sklearn import datasets
import io
from google.colab import files 


uploadedFile = files.upload()

Saving magic04.data to magic04.data


In [21]:
colunas_v3 = ['fLength', 'fWidth', 'fSize', 'fConc', 'fConc1', 'fAsym', 'fM3Long', 'fM3Trans', 'fAlpha', 'fDist', 'class']
df_v3 = pd.read_csv(io.StringIO(uploadedFile['magic04.data'].decode('utf-8')),names=colunas_v3,header=None) 
#df_v3 = pd.read_csv(io.StringIO(uploadedFile['magic04.data'].decode('utf-8')),header=None)

linhas, colunas = df_v3.shape
tamanho_treino = int(np.ceil(linhas*.80))

xtreino = df_v3.iloc[:tamanho_treino,1:colunas-1]
ytreino = df_v3.iloc[:tamanho_treino,colunas-1:]
xteste = df_v3.iloc[tamanho_treino:,1:colunas-1]
yteste = df_v3.iloc[tamanho_treino:,colunas-1:]

df_v3.head(5)

,fLength,fWidth,fSize,fConc,fConc1,fAsym,fM3Long,fM3Trans,fAlpha,fDist,class
0,28.7967,16.0021,2.6449,0.3918,0.1982,27.7004,22.0110,-8.2027,40.0920,81.8828,g
1,31.6036,11.7235,2.5185,0.5303,0.3773,26.2722,23.8238,-9.9574,6.3609,205.2610,g
2,162.0520,136.0310,4.0612,0.0374,0.0187,116.7410,-64.8580,-45.2160,76.9600,256.7880,g
3,23.8172,9.5728,2.3385,0.6147,0.3922,27.2107,-6.4633,-7.1513,10.4490,116.7370,g
4,75.1362,30.9205,3.1611,0.3168,0.1832,-5.5277,28.5525,21.8393,4.6480,356.4620,g


#REDE NEURAL: FEEDFORWARD

In [18]:
from keras.models import Sequential 
from keras.layers.core import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix

features = np.size(xtreino,1)

def criar_modelo():
  model = Sequential()
  model.add(Dense(50, input_dim=features, activation='sigmoid'))
  model.add(Dense(30, activation='sigmoid'))
  model.add(Dense(1, activation='sigmoid'))
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model

rede_neural_keras = KerasClassifier(build_fn=criar_modelo,verbose=0)

rede_neural_keras.fit(xtreino, ytreino, epochs=5, batch_size=10)

previstos_keras = rede_neural_keras.predict(xteste)

confusion_matrix(yteste,previstos_keras)

array([[   0,    0],
       [2124, 1680]])

#REDE NEURAL PARTE 1

In [19]:
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import confusion_matrix

rede_neural_mlp = MLPClassifier(solver='adam',alpha=1e-5,hidden_layer_sizes=(50,30,2),activation='logistic',batch_size=10,random_state=1,verbose=0)

rede_neural_mlp.fit(xtreino,ytreino.values.ravel())
previstos_mlp = rede_neural_mlp.predict(xteste)

confusion_matrix(yteste,previstos_mlp)

array([[   0,    0],
       [2096, 1708]])